In [4]:
import pandas as pd

# Corregir la ruta del archivo
df = pd.read_parquet("API/historico_ventas_1_año.parquet")
df_p = pd.read_parquet("API/costo_productos.parquet")


FileNotFoundError: [Errno 2] No such file or directory: 'API/historico_ventas_1_año.parquet'

In [ ]:
import re

# Diccionario de códigos a sucursales
mapa_codigos = {
    "FCAL": "CALI",
    "FMDE": "MEDELLIN",
    "FBOG": "BOGOTA",
    "FCTG": "CARTAGENA",
    "FBAQ": "BARRANQUILLA",
    "FVAL":"VALLADOLID"
}

def extraer_sucursal(nombre):
    if not isinstance(nombre, str):
        return "VENDEDOR EXTERNO"
    
    # 1) Buscar "Mostrador ..."
    match = re.search(r"Mostrador\s+([A-Za-z0-9\s]+)", nombre, re.IGNORECASE)
    if match:
        sucursal = match.group(1).strip().upper()
    else:
        # 2) Buscar "Calle" o "Cota"
        match2 = re.search(r"(Calle\s+\d+|Cota)", nombre, re.IGNORECASE)
        if match2:
            sucursal = match2.group(1).strip().upper()
        else:
            # 3) Buscar prefijo de código (ej: FCAL, FMDE, etc.)
            for prefijo, ciudad in mapa_codigos.items():
                if nombre.upper().startswith(prefijo):
                    return ciudad
            return "VENDEDOR EXTERNO"
    
    # 🔹 Limpiar T1, T2, T3
    sucursal = re.sub(r"\s*T\d+$", "", sucursal).strip()
    
    return sucursal

# Aplicar
df["Sucursal"] = df["invoice_name"].apply(extraer_sucursal)

In [ ]:
df = pd.merge(df,df_p,on='product_name',how="left")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1773194 entries, 0 to 1773193
Data columns (total 10 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   price_subtotal           float64       
 1   id                       int64         
 2   quantity                 float64       
 3   product_id_num           int64         
 4   product_name             object        
 5   invoice_id_num           int64         
 6   invoice_name             object        
 7   date_invoice             datetime64[ns]
 8   Sucursal                 object        
 9   producto_costo_unitario  float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(3)
memory usage: 135.3+ MB


In [ ]:
import pandas as pd
from datetime import date

# Semana y año actual
hoy = date.today()
semana_actual = 34  # lo defines manual si quieres
año_actual = hoy.year



# ================================
# 2) Agrupar ventas semanales
# ================================
df["Año"] = df["date_invoice"].dt.year
df["Semana"] = df["date_invoice"].dt.isocalendar().week

df_semana = (
    df.groupby(["Sucursal", "product_name", "Año", "Semana"])
      .agg({
          "quantity": "sum",
          "price_subtotal": "sum",
          "producto_costo_unitario": "mean"
      })
      .reset_index()
      .sort_values(["Sucursal", "product_name", "Año", "Semana"]) 
)

# ================================
# 3) Calcular ponderación personalizada
# ================================
# Asignar pesos: semanas 1-8 (80%), semanas 9-12 (20%)
df_semana = df_semana.sort_values(["Sucursal", "product_name", "Semana"])

def ema_custom(x):
    n = len(x)
    if n < 12:
        return None  # Si no hay 12 semanas completas
    ultimas = x.iloc[-4:]   # semanas 9-12
    anteriores = x.iloc[:-4]  # semanas 1-8
    return (ultimas.mean() * 0.2) + (anteriores.mean() * 0.8)

df_resultado = (
    df_semana.groupby(["Sucursal", "product_name"], as_index=False)
             .agg(EMA=("quantity", ema_custom),
                  Costo_unitario=("producto_costo_unitario", "mean"))
)


pd.set_option("display.max_colwidth", None) #muestra todo el contenido

# ================================
# 4) Agregar semana y año actuales
# ================================
df_resultado["Semana"] = semana_actual
df_resultado["Año"] = año_actual

# ================================
# 5) Calcular "Venta costo EMA"
# ================================
df_resultado["Venta_EMA"] = df_resultado["EMA"] * df_resultado["Costo_unitario"]

# ================================
# 6) Dejar columnas estilo Odoo
# ================================
df_resultado = df_resultado[[
    "Sucursal", "product_name", "Año", "Semana", "EMA",
    "Costo_unitario", "Venta_EMA"

]]


In [ ]:
df_resultado.sample(5)

,Sucursal,product_name,Año,Semana,EMA,Costo_unitario,Venta_EMA
9740,NORTE,[DAR02044025] DA2044 FILTRO AIRE CATERPILLAR (025 DA2044),2025,34,NaN,42579.89,NaN
2371,BUCARAMANGA,[DAR08132025] DA8132 FILTRO AIRE 1° YUTONG ZK6858 (025 DA8132),2025,34,NaN,9791.78,NaN
4677,CALLE 6,[DAB08120025] DA8120 FILTRO AIRE - VOLVO (025 DA8120),2025,34,NaN,74699.71,NaN
10322,VALLADOLID,[BCS00696125] GS696 FILTRO COMBUSTIBLE NGERSOLL RAND 185 AIR SOURCE PLUS (125 BF46040D),2025,34,NaN,56661.78,NaN
12752,VENDEDOR EXTERNO,[DAP02020025] DA2020 FILTRO AIRE DONSSON - RETROCARGADORES CATERPILLAR (025 DA2020),2025,34,197.989535,25852.39,5.118503e+06


In [ ]:
df_resultado = df_resultado.sort_values(by=("EMA"),ascending=False)

sede ="VALLADOLID"
df_sede = df_resultado[(df_resultado['Sucursal']==sede) & (df_resultado["product_name"].str.contains("GRASA"))]
df_sede

,Sucursal,product_name,Año,Semana,EMA,Costo_unitario,Venta_EMA
10040,VALLADOLID,[AGBGRASA030] GRASA MOBILUX EP 2 X GRAMOS (030 GRASA),2025,34,NaN,26.84,NaN
